<a href="https://colab.research.google.com/github/nikhilveldanda/INFO-5731/blob/main/Assignment3/INFO5731_Assignment_Three.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Three**

In this assignment, you are required to conduct information extraction, semantic analysis based on **the dataset you collected from assignment two**. You may use scipy and numpy package in this assignment. (Use 500 records amount all the data you collected).

# **Question 1: Understand N-gram**

(45 points). Write a python program to conduct N-gram analysis based on the dataset in your assignment two:

(1) Count the frequency of all the N-grams (N=3).

(2) Calculate the probabilities for all the bigrams in the dataset by using the fomular count(w2 w1) / count(w2). For example, count(really like) / count(really) = 1 / 3 = 0.33.

(3) Extract all the **noun phrases** and calculate the relative probabilities of each review in terms of other reviews (abstracts, or tweets) by using the fomular frequency (noun phrase) / max frequency (noun phrase) on the whole dataset. Print out the result in a table with column name the all the noun phrases and row name as the 500 reviews (abstracts, or tweets). 


In [1]:
!pip install selenium
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)



Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromium-chromedriver is already the newest version (95.0.4638.69-0ubuntu0.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 47 not upgraded.
cp: '/usr/lib/chromium-browser/chromedriver' and '/usr/bin/chromedriver' are the same file


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: use options instead of chrome_options
  # This is added back by InteractiveShellApp.init_path()


In [2]:
import pandas as pd 
# Import Selenium and its sub libraries
import selenium 
from selenium import webdriver
# Import BS4
import requests
from bs4 import BeautifulSoup
from pandas.core.frame import DataFrame
import re


/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [3]:
import nltk
nltk.download()


NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all
       | 
       | Downloading package abc to /root/nltk_data...
       |   Package abc is already up-to-date!
       | Downloading package alpino to /root/nltk_data...
       |   Package alpino is already up-to-date!
       | Downloading package biocreative_ppi to /root/nltk_data...
       |   Package biocreative_ppi is already up-to-date!
       | Downloading package brown to /root/nltk_data...
       |   Package brown is already up-to-date!
       | Downloading package brown_tei to /root/nltk_data...
       |   Package brown_tei is already up-to-date!
       | Downloading package cess_cat to /root/nltk_data...
       |   Package cess_cat is already up-to-date!
       | Down

True

In [4]:
def get_reviews(url) -> DataFrame:
  driver.get(url)
  driver.implicitly_wait(1) 
  # Set up action to click on 'load more' button
  page = 1
  while page<5:  
      try:
          load_more = driver.find_element_by_id('load-more-trigger')
          load_more.click()
          page+=1 
      except:
          break
  # After fully expand the page, we will grab data from whole website
  review = driver.find_elements_by_class_name('review-container')

  title = []
  content = []
  for n in range(len(review)):
      try:
          #Some reviewers only give review text or rating without the other, 
          #so we use try/except here to make sure each block of content must has all the element before append them to the list
          #Check if each review has all the elements
          ftitle = review[n].find_element_by_class_name('title').text
          fcontent = review[n].find_element_by_class_name('content').get_attribute("textContent").strip()

          #Then add them to the respective list
          title.append(ftitle)
          content.append(fcontent)
      except:
          continue
  #Build data dictionary for dataframe
  data = {
          'title': title,
          'text' : content
          
      }
  #Build dataframe for each movie to export
  review = pd.DataFrame(data = data)
  return review

In [5]:
df = get_reviews("https://www.imdb.com/title/tt6751668/reviews?ref_=tt_urv")
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  
/usr/local/lib/python3.7/dist-packages/selenium/webdriver/remote/webelement.py:445: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")


,title,text
0,I loved the first 90%....but the last might be...,"I was able to see ""Parasite"" a few days ago at..."
1,Thoroughly entertaining,PARASITE is an excellent effort from Korean di...
2,class warfare,The Kim family leeches off others and lives in...
3,Totally Unique,Usually I don't read reviews before seeing a f...
4,"Fairly entertaining film, but one of 2019's be...","Okay, so these slightly over two hours are ""Pa..."


In [6]:
df

,title,text
0,I loved the first 90%....but the last might be...,"I was able to see ""Parasite"" a few days ago at..."
1,Thoroughly entertaining,PARASITE is an excellent effort from Korean di...
2,class warfare,The Kim family leeches off others and lives in...
3,Totally Unique,Usually I don't read reviews before seeing a f...
4,"Fairly entertaining film, but one of 2019's be...","Okay, so these slightly over two hours are ""Pa..."
...,...,...
95,Parasite (Gisaengchung),10/10 - undoubtedly original and captivating.....
96,This is just a well done film that is very com...,Parasite (2019) is a movie we saw in theatres ...
97,"As Stakes Rapidly Rise, a Clever Family of Con...","A relatively simple, low-key racket spirals wa..."
98,"Parasite Review: Comedy, drama, suspense, and ...",A family who doesn't have a lot of money or op...


In [7]:
from textblob import Word

import nltk
nltk.download('wordnet')
from nltk.stem import PorterStemmer
st = PorterStemmer()
import nltk
nltk.download('punkt')
from textblob import TextBlob
from nltk.corpus import stopwords
stop = stopwords.words('english')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
df['Lower Case'] = df['text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df['Removal of Punctuation'] = df['Lower Case'].str.replace('[^\w\s]','')
df['Removal of Special Characters'] = df['Removal of Punctuation'].apply(lambda x: ''.join(re.sub(r"[^a-zA-Z0-9]+", ' ', charctr) for charctr in x ))

df['Stopwords Removal'] = df['Removal of Punctuation'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
df['Spelling Correction'] = df['Stopwords Removal'].apply(lambda x: str(TextBlob(x).correct()))

df['Tokenization'] = df['Spelling Correction'].apply(lambda x: TextBlob(x).words)

df['After Stemming'] = df['Tokenization'].apply(lambda x: " ".join([st.stem(word) for word in x]))

df['After Lemmatization'] = df['After Stemming'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
df.to_csv('/content/sample_data/reviwes.csv',index=False)


1.1 Trigrams

In [9]:
import itertools
import collections
from nltk import ngrams
from nltk.tokenize import word_tokenize

words_sentence = []
for sentence in df['After Lemmatization']:
  words_sentence.append(word_tokenize(sentence))
aftercleaning = [x for x in words_sentence if x != []]
iterations = list(itertools.chain.from_iterable(aftercleaning))
tri_grams = nltk.trigrams(iterations)
frequency_dist = nltk.FreqDist(tri_grams)
frequency_dist

FreqDist({('abl', 'see', 'parasit'): 1,
          ('see', 'parasit', 'day'): 1,
          ('parasit', 'day', 'ago'): 1,
          ('day', 'ago', 'philadelphia'): 1,
          ('ago', 'philadelphia', 'film'): 1,
          ('philadelphia', 'film', 'festiv'): 1,
          ('film', 'festiv', 'mani'): 1,
          ('festiv', 'mani', 'peopl'): 1,
          ('mani', 'peopl', 'love'): 1,
          ('peopl', 'love', 'film'): 1,
          ('love', 'film', 'daughter'): 1,
          ('film', 'daughter', 'saw'): 1,
          ('daughter', 'saw', 'think'): 1,
          ('saw', 'think', 'product'): 1,
          ('think', 'product', 'valu'): 1,
          ('product', 'valu', 'excel'): 1,
          ('valu', 'excel', 'end'): 1,
          ('excel', 'end', 'chang'): 1,
          ('end', 'chang', 'overal'): 1,
          ('chang', 'overal', 'feel'): 1,
          ('overal', 'feel', 'movi'): 1,
          ('feel', 'movi', 'until'): 1,
          ('movi', 'until', 'end'): 1,
          ('until', 'end', 'like'): 1,


1.2 Bigrams probability

In [10]:
from collections import Counter
bi_grams = nltk.bigrams(iterations)
frequency_dist = nltk.FreqDist(bi_grams)
bi_grams_dict = dict(frequency_dist)
for word in bi_grams_dict:
  print( str(word) + ':' + str(bi_grams_dict[word] / iterations.count(word[0])))


Streaming output truncated to the last 5000 lines.
('storytel', 'gener'):0.3333333333333333
('gener', '5'):0.125
('5', '13'):0.14285714285714285
('13', 'found'):0.5
('permalink', 'hard'):0.024390243902439025
('hard', 'nail'):0.1111111111111111
('nail', 'type'):1.0
('type', 'movi'):0.5
('movi', 'parasit'):0.007936507936507936
('parasit', 'dramathrillercomedi'):0.008333333333333333
('dramathrillercomedi', 'definit'):1.0
('worth', 'see'):0.1
('see', 'long'):0.02040816326530612
('joonho', 'tell'):0.03225806451612903
('stori', 'way'):0.016129032258064516
('way', 'keep'):0.016666666666666666
('keep', 'engag'):0.05555555555555555
('engag', 'keep'):0.07692307692307693
('keep', 'guess'):0.05555555555555555
('guess', 'protagonist'):0.16666666666666666
('protagonist', 'can'):0.2
('can', 'fulli'):0.047619047619047616
('fulli', 'like'):0.16666666666666666
('like', 'antagonist'):0.011764705882352941
('antagonist', 'dont'):0.25
('dont', 'realli'):0.03333333333333333
('realli', 'hate'):0.04
('hate', '

1.3

In [11]:

whole_dataset = ''
index = []
i = 1
for line in df['After Lemmatization']:
  whole_dataset = whole_dataset + line
  value = 'Review-' + str(i)
  index.append(value)
  i += 1

from textblob import TextBlob
noun_phrases = []
frequency = []
for line in df['After Lemmatization']:
  blob = TextBlob(line)
  for nouns in blob.noun_phrases:
    noun_phrases.append(nouns)
for word in noun_phrases:
  noun_phrases_freq = []
  for line in df['After Lemmatization']:
    noun_phrases_freq.append(line.count(word) / whole_dataset.count(word))
  frequency.append(noun_phrases_freq)
noun_phrases_df = pd.DataFrame(frequency).T
noun_phrases_df.columns = list(noun_phrases)
noun_phrases_df.index = index
noun_phrases_df


,parasit day,philadelphia film festiv mani peopl,film daughter,product valu excel end chang overal,muchbut ultraviol climax,cold experi film realli,violenc impart youinstead talk plot,great film,focu violenc daughter,extrem need plot,knive plung peopl,close rang,giant rock smash ultrarealist person head,extrem person closeupso overal,good film,parasit excel effort organ director,ho talent,admir sinc,host decad,perhap tower masterpiec,quiet effect work matur,payabl play look class divid scath extrem,way issu univers film,champion qualiti,organ cinema,long time technic qualiti exemplari act particularli astound unusu storylin,unexpect climax film thoroughli,famili leech,son kiwoo,job tutor,young girl use seri lie member,rich park famili home,famili annoy,penza box incid remind,real life peopl imposs work frustrat,coupl tri,style famili annoy penza girl,tune movi continu,horni need complet saint fact,rich famili need,...,messag movi titl movi,thing head,ah experi parasit,class societyfrom technic aspect,direct cinematographi mani beauti scene movi pretti,consid action,place insid,big hous,big garden favorit sequenc,famili run home rain,board titan,top engin room wonder sequenc beauti watchth cast fantast everyon,fit need role charact wellwritten,complement perfectli,great job creat,real peopl despit eccentr,ground realist feelth music score complement scene,nice elev emot respons,foreign film tiller,littl check,crisp film technic aspect blend,wrong solid film,audienc intellig,famili liter,depth catch,employ tutor wealthi famili crafti famili conspir inveigh way household part,chauffeur longtim housekeep fire,lavish domicil garden eden front yard thing quickli,howev housekeep return someth,class defin,plot slowli unfold darkli,comic fashion,self induc chronic,free fall plausibl absurd hurri climax satisfi epilogu director,hope cinematograph,excel job contrast,situat jaw class charact earli,equal divid role adult charact benign petti way,fascin slightli balanc plot,end game plot hope decid end conflagr insipid post climax demand defi logic
Review-1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.25,1.0,1.0,1.0,1.0,1.0,1.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Review-2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.25,1.0,1.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Review-3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,1.0,0.125,0.333333,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Review-4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Review-5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

# **Question 2: Undersand TF-IDF and Document representation**

(40 points). Starting from the documents (all the reviews, or abstracts, or tweets) collected for assignment two, write a python program: 

(1) To build the **documents-terms weights (tf*idf) matrix bold text**.

(2) To rank the documents with respect to query (design a query by yourself, for example, "An Outstanding movie with a haunting performance and best character development") by using **cosine similarity**.

In [12]:
# Write your code here


import math
def tf_idf_value(sentence,word):
  size = len(sentence.split(" "))
  tf_value = sentence.count(word)/size
  idf_value = 0
  if(tf_value!=0):
    idf_value = math.log(size)/sentence.count(word)
  else:
    return 0;
  return tf_value*idf_value

sentences = df["Spelling Correction"].values.tolist()
tokens_list = set([j for i in sentences for j in i.split(" ")])
tf_idf = pd.DataFrame(tokens_list,columns=["token"])
count = 0
for i in sentences:
  tf_idf[str(count)] = tf_idf["token"].apply(lambda x : tf_idf_value(i,x))
  count+=1

tf_idf



,token,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,underhanded,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.018358,0.00000,0.000000,0.0,0.0,0.0,0.00000,0.000000,0.00000,0.047936,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.03125,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0
1,helpful,0.0,0.000000,0.0,0.079425,0.0,0.047162,0.000000,0.0,0.0,0.056983,0.06348,0.000000,0.05374,0.000000,0.0,0.0,0.0,0.06348,0.000000,0.08065,0.000000,0.095726,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.000000,0.055855,0.045343,0.04304,0.051795,0.050879,0.0,...,0.070008,0.0,0.000000,0.056413,0.000000,0.081918,0.077095,0.0,0.0,0.000000,0.058773,0.0,0.000000,0.062052,0.000000,0.099542,0.0,0.065764,0.110774,0.136326,0.079425,0.0,0.0,0.000000,0.0,0.0,0.072861,0.0,0.000000,0.066567,0.065764,0.000000,0.059398,0.040702,0.0,0.180537,0.0,0.043671,0.000000,0.0
2,toilet,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.015105,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0
3,here,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.023232,0.0,0.0,0.000000,0.00000,0.018358,0.00000,0.000000,0.0,0.0,0.0,0.00000,0.017313,0.00000,0.000000,0.000000,0.049998,0.0,0.0,0.0,0.0,0.030354,0.00000,0.0,0.030211,0.0,0.000000,0.000000,0.045343,0.00000,0.000000,0.000000,0.0,...,0.000000,0.0,0.026929,0.056413,0.043671,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.015105,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.032879,0.0,0.0,0.000000,0.0,0.012244,0.000000,0.000000,0.059398,0.000000,0.040702,0.0,0.000000,0.0,0.000000,0.022682,0.0
4,exemplary,0.0,0.056983,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4172,descend,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.00000,0.000000,0.00000,0.000000,0.000000,0.049998,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.030211,0.0,0.056413,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0

2.2


In [13]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
query_sentence = """An Outstanding movie with a haunting performance and best character development"""
X_list = word_tokenize(query_sentence)
sw = stopwords.words('english') 
X_set = {w for w in X_list if not w in sw}


In [14]:
def compute_vector(X_set, Y_set):
  rvector = X_set.union(Y_set)
  l1 =[];l2 =[]
  for w in rvector: 
      if w in X_set: l1.append(1)
      else: l1.append(0) 
      if w in Y_set: l2.append(1) 
      else: l2.append(0)
  return rvector, l1, l2

def compute_cosine(rvector, l1, l2):
  c = 0
  for i in range(len(rvector)): 
        c+= l1[i]*l2[i] 
  cosine = c / float((sum(l1)*sum(l2))**0.5) 
  return cosine

similarity_list = []
for sentence in df['Spelling Correction']:
  Y_list = word_tokenize(sentence)
  Y_set = {w for w in Y_list if not w in sw}
  rvector, l1, l2 = compute_vector(X_set, Y_set)
  try:
    similarity = compute_cosine(rvector, l1, l2)
  except ZeroDivisionError:
    similarity = 'None'
  similarity_list.append(similarity)
cosine_dataframe = pd.DataFrame(list(zip(df['text'],similarity_list)), columns=['text','Cosine Similarity'])
cosine_dataframe

,text,Cosine Similarity
0,"I was able to see ""Parasite"" a few days ago at...",0.042563
1,PARASITE is an excellent effort from Korean di...,0.000000
2,The Kim family leeches off others and lives in...,0.036662
3,Usually I don't read reviews before seeing a f...,0.000000
4,"Okay, so these slightly over two hours are ""Pa...",0.037851
...,...,...
95,10/10 - undoubtedly original and captivating.....,0.000000
96,Parasite (2019) is a movie we saw in theatres ...,0.106066
97,"A relatively simple, low-key racket spirals wa...",0.035007
98,A family who doesn't have a lot of money or op...,0.026880


# **Question 3: Create your own training and evaluation data for sentiment analysis**

(15 points). **You dodn't need to write program for this question!** Read each of the 500 review (if you collected the data from citeseer, you can think of other text classfication tasks instead of sentiment analysis) you collected in detail, and annotate each review with a sentiment (positive, negative, or neutral). Save the annotated dataset into a csv file with three columns (first column: document_id, clean_text, sentiment), upload the csv file to GitHub and submit the file link blew. This datset will be used for assignment four: sentiment analysis and text classification. 


In [15]:
# The GitHub link of your final csv file

# Link: 

"""https://github.com/nikhilveldanda/INFO-5731/blob/main/Assignment3/Reviews_analysis.csv"""